In [3]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator
from collections import Counter
import dateparser

## Отчет по долям рынка

https://cbr.ru/statistics/bbs/statisticheskiy-byulleten-banka-rossii/

По этому адресу статистический бюллетень ЦБ. Нужны показатели по регионам (там они приведенные к доллару, поэтому мы наши евро умножаем на 1,15)


https://cbr.ru/analytics/podft/currency_cash/

По этому адресу основные показатели наличного рынка валюты. Оттуда берем совокупные показатели по рынку в $ и €.

In [4]:
FOLDER = r'\\tsclient\X\УМиР\2021\аналитика\Доли рынка'

os.chdir(FOLDER)

## Загрузим справочник

In [5]:
directory = pd.read_excel(r'\\tsclient\X\УМиР\Аманов\Справочник_филиалы.xlsx')

### Обработаем ежедневные отчеты по банку (сохраняю за целый месяц в формате %m-%Y)

In [6]:
pattern = r'^\d{2}-\d{4}\.xlsx$'

indicators = ['Доход от валютно-обменных операций', 'Приход USD', 'Расход USD', 'Приход EUR', 'Расход EUR', 'Количество валютно-обменных операций', 'Остаток в кассе в нац. эквиваленте']

for file in os.listdir():
    if re.match(pattern, file):
        temp_df = pd.read_excel(file)
        temp_df = temp_df[[col for col in temp_df.columns if 'unnamed' not in str(col).lower()]]
        temp_df = temp_df.merge(directory[['Ежедневный отчет', 'Город']], left_on='СВОД', right_on='Ежедневный отчет', how='left')
        temp_df.drop('Ежедневный отчет', axis=1, inplace=True)
        temp_df = temp_df.dropna(subset=['СВОД'])
        temp_df = temp_df.reset_index(drop=True)
        indicators_rows_number = {element: temp_df['СВОД'][temp_df['СВОД'] == element].index[0] for element in indicators}
        new_values = list(zip(list(indicators_rows_number.values())[:-1], list(indicators_rows_number.values())[1:]))
        for key, value in indicators_rows_number.items():
            for i in new_values:
                if value in i:
                    indicators_rows_number[key] = i
        indicators_rows_number.pop('Остаток в кассе в нац. эквиваленте')
        profit, usd_income, usd_outcome, euro_income, euro_outcome, quant = [temp_df.iloc[i[0]:i[1], :] for i in indicators_rows_number.values()]
        all_tables = [profit, usd_income, usd_outcome, euro_income, euro_outcome, quant]
        for table in all_tables:
            table.reset_index(drop=True, inplace=True)
            new_name = table['СВОД'][0]
            table.rename(columns={'СВОД': new_name}, inplace=True)
            table.dropna(subset=['Город'], inplace=True)
        indicators_to_excel = {'весь рынок приход, млн $ Заубер Банк': '',
                              'весь рынок расход, млн $ Заубер Банк': '',
                              'весь рынок приход, млн евро (в $) Заубер Банк': '',
                              'весь рынок расход, млн евро (в $) Заубер Банк': '',
                              'Мск+СПб приход Заубер Банк млн евро+$': '',
                              'Мск+СПб приход Заубер Банк млн евро  (в $)': '',
                              'Мск+СПб приход Заубер Банк млн $': '',
                              'Мск+СПб расход Заубер Банк млн евро+$': '',
                              'Мск+СПб расход Заубер Банк млн евро  (в $)': '',
                              'Мск+СПб расход Заубер Банк млн $': '',
                              'Мск приход Заубер Банк млн евро+$': '',
                              'Мск приход Заубер Банк млн евро  (в $)': '',
                              'Мск приход Заубер Банк млн $': '',
                              'Мск расход Заубер Банк млн евро+$': '',
                              'Мск расход Заубер Банк млн евро  (в $)': '',
                              'Мск расход Заубер Банк млн $': '',
                              'СПб приход Заубер Банк млн евро+$': '',
                              'СПб приход Заубер Банк млн евро  (в $)': '',
                              'СПб приход Заубер Банк млн $': '',
                              'СПб расход Заубер Банк млн евро+$': '',
                              'СПб расход Заубер Банк млн евро  (в $)': '',
                              'СПб расход Заубер Банк млн $': '',
                              'весь рынок Количество сделок, тыс ед. Заубер Банк': '',
                              'весь рынок Средний размер сделки, ед Заубер Банк': ''
                              }
        usd_income_cities = usd_income.groupby('Город').sum().sum(axis=1) / 1000000
        usd_outcome_cities = usd_outcome.groupby('Город').sum().sum(axis=1) / 1000000
        euro_income_cities = (euro_income.groupby('Город').sum().sum(axis=1) / 1000000) * 1.15
        euro_outcome_cities = (euro_outcome.groupby('Город').sum().sum(axis=1) / 1000000) * 1.15


        indicators_to_excel['весь рынок приход, млн $ Заубер Банк'] = usd_income_cities.sum()
        indicators_to_excel['весь рынок расход, млн $ Заубер Банк'] = usd_outcome_cities.sum()
        indicators_to_excel['весь рынок приход, млн евро (в $) Заубер Банк'] = euro_income_cities.sum()
        indicators_to_excel['весь рынок расход, млн евро (в $) Заубер Банк'] = euro_outcome_cities.sum()
        indicators_to_excel['Мск+СПб приход Заубер Банк млн евро+$'] = usd_income_cities.sum() + euro_income_cities.sum()
        indicators_to_excel['Мск+СПб приход Заубер Банк млн евро  (в $)'] = euro_income_cities.sum()
        indicators_to_excel['Мск+СПб приход Заубер Банк млн $'] = usd_income_cities.sum()
        indicators_to_excel['Мск+СПб расход Заубер Банк млн евро+$'] = usd_outcome_cities.sum() + euro_outcome_cities.sum()
        indicators_to_excel['Мск+СПб расход Заубер Банк млн евро  (в $)'] = euro_outcome_cities.sum()
        indicators_to_excel['Мск+СПб расход Заубер Банк млн $'] = usd_outcome_cities.sum()
        indicators_to_excel['Мск приход Заубер Банк млн евро+$'] = usd_income_cities['Москва'] + euro_income_cities['Москва']
        indicators_to_excel['Мск приход Заубер Банк млн евро  (в $)'] = euro_income_cities['Москва']
        indicators_to_excel['Мск приход Заубер Банк млн $'] = usd_income_cities['Москва']
        indicators_to_excel['Мск расход Заубер Банк млн евро+$'] = usd_outcome_cities['Москва'] + euro_outcome_cities['Москва']
        indicators_to_excel['Мск расход Заубер Банк млн евро  (в $)'] = euro_outcome_cities['Москва']
        indicators_to_excel['Мск расход Заубер Банк млн $'] = usd_outcome_cities['Москва']
        indicators_to_excel['СПб приход Заубер Банк млн евро+$'] = usd_income_cities['Санкт-Петербург'] + euro_income_cities['Санкт-Петербург']
        indicators_to_excel['СПб приход Заубер Банк млн евро  (в $)'] = euro_income_cities['Санкт-Петербург']
        indicators_to_excel['СПб приход Заубер Банк млн $'] = usd_income_cities['Санкт-Петербург']
        indicators_to_excel['СПб расход Заубер Банк млн евро+$'] = usd_outcome_cities['Санкт-Петербург'] + euro_outcome_cities['Санкт-Петербург']
        indicators_to_excel['СПб расход Заубер Банк млн евро  (в $)'] = euro_outcome_cities['Санкт-Петербург']
        indicators_to_excel['СПб расход Заубер Банк млн $'] = usd_outcome_cities['Санкт-Петербург']
        indicators_to_excel['весь рынок Количество сделок, тыс ед. Заубер Банк'] = quant.sum(axis=1).sum() / 1000
        indicators_to_excel['весь рынок Средний размер сделки, ед Заубер Банк'] = ((indicators_to_excel['Мск+СПб приход Заубер Банк млн евро+$'] + indicators_to_excel['Мск+СПб расход Заубер Банк млн евро+$']) * 1000000) / (indicators_to_excel['весь рынок Количество сделок, тыс ед. Заубер Банк'] * 1000)
        
        pd.Series(indicators_to_excel).to_excel(f"{file[:file.find('.')]}_показатели_банковские.xlsx")
        
        
        



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-7fa8fa1cbcd3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.dropna(subset=['Город'], inplace=True)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-7fa8fa1cbcd3>:26: 